In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

from BayesianOptimization.bayesian_optimization import BayesianOptimization

from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from EMV_RSIStrategy import EMV_RSIStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, window_EMV=40, n=10, m=10, window_RSI = 10, s=70, b=30):
    window_EMV = int(window_EMV)
    window_RSI = int(window_RSI)
    n = int(n)
    m = int(m)
    s = int(s)
    b = int(b)
    config['title'] = "EMV_RSIStrategy" + "_" + str(window_EMV) + "_" + str(n) + "_" + str(m) + "_" + str(window_RSI) + "_" + str(s) + "_" + str(b)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = EMV_RSIStrategy(config, events_queue, data_handler,
                           window_EMV=window_EMV, n=n, m=m,
                            window_RSI=window_RSI, s=s, b=b)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/EMV_RSIStrategy",
    "title": "EMV_RSIStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [6]:
BO = BayesianOptimization(
    run_backtest,
    {'window_EMV': (1, 120),
     'n': (1, 120),
      'm': (1, 120),
    'window_RSI': (1, 120), 
       's': (55, 85),
      'b': (15, 45)},
    is_int = [1, 1, 1, 1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=100, n_iter=150, acq="ei", xi=0.01, **gp_params)

Initialization
-----------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |         b |         m |         n |         s |   window_EMV |   window_RSI | 
---------------------------------
EMV_RSIStrategy_38_24_56_48_62_40
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.7338949622
Sortino Ratio: -0.9601061531
Max Drawdown: 0.3578575406
Max Drawdown Duration: 135000
Total Returns: -0.2908611210
Annualized Returns: -0.6845629824
Compound Annual Growth Rate: -0.5546667135
---------------------------------
Trades: 8
Trade Winning: 50.00%
Average Trade: -3.40%
Average Win: 4.28%
Average Loss: -11.08%
Best Trade: 9.24%
Worst Trade: -26.46%
Worst

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.5545977255
Sortino Ratio: 0.2818945243
Max Drawdown: 0.4003503599
Max Drawdown Duration: 227600
Total Returns: 0.0669427022
Annualized Returns: 0.1575545596
Compound Annual Growth Rate: 0.1647484371
---------------------------------
Trades: 4
Trade Winning: 50.00%
Average Trade: 2.79%
Average Win: 15.89%
Average Loss: -10.31%
Best Trade: 16.80%
Worst Trade: -16.28%
Worst Trade Date: 3 days 08:00:00
Avg Days in Trade: 3 days 06:45:00
---------------------------------
   10 | 00m10s |    0.06694 | 41.000000 | 114.000000 | 63.000000 | 71.000000 | 17.000000 | 68.000000 | 
---------------------------------
EMV_RSIStrategy_2_47_15_25_84_25
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
B

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.5296220415
Sortino Ratio: nan
Max Drawdown: 0.0012299740
Max Drawdown Duration: 200
Total Returns: 0.0269763395
Annualized Returns: 0.0634907937
Compound Annual Growth Rate: 0.0646536290
---------------------------------
Trades: 1
Trade Winning: 100.00%
Average Trade: 2.90%
Average Win: 2.90%
Average Loss: nan%
Best Trade: 2.90%
Worst Trade: 2.90%
Worst Trade Date: 0 days 05:00:00
Avg Days in Trade: 0 days 05:00:00
---------------------------------
   19 | 00m26s |    0.02698 | 24.000000 | 60.000000 | 4.000000 | 64.000000 | 102.000000 | 80.000000 | 
---------------------------------
EMV_RSIStrategy_19_1_16_64_75_28
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   28 | 00m21s |    0.00000 | 19.000000 | 102.000000 | 92.000000 | 75.000000 | 69.000000 | 99.000000 | 
---------------------------------
EMV_RSIStrategy_88_22_22_31_62_38
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Rati

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   37 | 00m28s |    0.00000 | 21.000000 | 41.000000 | 21.000000 | 62.000000 | 106.000000 | 40.000000 | 
---------------------------------
EMV_RSIStrategy_10_31_79_36_61_41
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Rati

...

---------------------------------
EMV_RSIStrategy_22_5_105_25_82_36
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.5437921404
Sortino Ratio: -0.3033247773
Max Drawdown: 0.2536102915
Max Drawdown Duration: 185900
Total Returns: -0.1020174868
Annualized Returns: -0.2401056380
Compound Annual Growth Rate: -0.2237304930
---------------------------------
Trades: 14
Trade Winning: 50.00%
Average Trade: -0.52%
Average Win: 1.85%
Average Loss: -2.90%
Best Trade: 4.47%
Worst Trade: -5.82%
Worst Trade Date: 0 days 07:00:00
Avg Days in Trade: 0 days 15:51:26
---------------------------------
  208 | 00m58s |   -0.10202 | 36.000000 | 105.000000 | 5.000000 | 82.000000 | 22.000000 | 25.000000 | 
--------------------------------

---------------------------------
EMV_RSIStrategy_59_91_85_84_79_39
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 3.9064111861
Sortino Ratio: 5.0773635104
Max Drawdown: 0.1596311927
Max Drawdown Duration: 132300
Total Returns: 1.0597542962
Annualized Returns: 2.4942094666
Compound Annual Growth Rate: 4.4775657890
---------------------------------
Trades: 5
Trade Winning: 100.00%
Average Trade: 16.34%
Average Win: 16.34%
Average Loss: nan%
Best Trade: 37.65%
Worst Trade: 6.75%
Worst Trade Date: 1 days 19:00:00
Avg Days in Trade: 3 days 16:36:00
---------------------------------
  217 | 01m11s |    1.05975 | 39.000000 | 85.000000 | 91.000000 | 79.000000 | 59.000000 | 84.000000 | 
---------------------------------
EMV_

---------------------------------
EMV_RSIStrategy_43_78_56_43_65_26
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.5296220415
Sortino Ratio: nan
Max Drawdown: 0.0010000000
Max Drawdown Duration: 100
Total Returns: 0.0038997333
Annualized Returns: 0.0091783084
Compound Annual Growth Rate: 0.0092025438
---------------------------------
Trades: 1
Trade Winning: 100.00%
Average Trade: 0.59%
Average Win: 0.59%
Average Loss: nan%
Best Trade: 0.59%
Worst Trade: 0.59%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 01:00:00
---------------------------------
  226 | 00m46s |    0.00390 | 26.000000 | 56.000000 | 78.000000 | 65.000000 | 43.000000 | 43.000000 | 
---------------------------------
EMV_RSIStrategy_1_1

  234 | 00m54s |    0.04362 | 45.000000 | 64.000000 | 84.000000 | 55.000000 | 9.000000 | 1.000000 | 
---------------------------------
EMV_RSIStrategy_62_79_9_31_79_16
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.3741304898
Sortino Ratio: nan
Max Drawdown: 0.0164094074
Max Drawdown Duration: 1000
Total Returns: 0.0492917715
Annualized Returns: 0.1160117998
Compound Annual Growth Rate: 0.1199042621
---------------------------------
Trades: 1
Trade Winning: 100.00%
Average Trade: 5.14%
Average Win: 5.14%
Average Loss: nan%
Best Trade: 5.14%
Worst Trade: 5.14%
Worst Trade Date: 1 days 11:00:00
Avg Days in Trade: 1 days 11:00:00
---------------------------------
  235 | 01m00s |    0.04929 | 16.000000 | 9.000000 | 79

---------------------------------
EMV_RSIStrategy_54_82_80_75_80_39
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.9239763275
Sortino Ratio: 0.8748171856
Max Drawdown: 0.3116794702
Max Drawdown Duration: 172400
Total Returns: 0.3480883003
Annualized Returns: 0.8192513463
Compound Annual Growth Rate: 1.0197686491
---------------------------------
Trades: 5
Trade Winning: 80.00%
Average Trade: 7.37%
Average Win: 13.05%
Average Loss: -15.34%
Best Trade: 23.29%
Worst Trade: -15.34%
Worst Trade Date: 1 days 06:00:00
Avg Days in Trade: 3 days 12:48:00
---------------------------------
  244 | 03m19s |    0.34809 | 39.000000 | 80.000000 | 82.000000 | 80.000000 | 54.000000 | 75.000000 | 
---------------------------------
E

In [7]:
print(BO.res['max'])

{'max_val': 1.059754296185353, 'max_params': {'window_EMV': 59.0, 'n': 91.0, 'm': 85.0, 'window_RSI': 84.0, 's': 79.0, 'b': 39.0}}


In [8]:
BO.maximize(init_points=0, n_iter=150, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
-----------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |         b |         m |         n |         s |   window_EMV |   window_RSI | 
---------------------------------
EMV_RSIStrategy_120_120_118_120_85_15
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Tr

---------------------------------
EMV_RSIStrategy_120_1_65_1_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.3710826193
Sortino Ratio: -1.2384635617
Max Drawdown: 0.5020958311
Max Drawdown Duration: 293900
Total Returns: -0.3922979340
Annualized Returns: -0.9233019618
Compound Annual Growth Rate: -0.6903292208
---------------------------------
Trades: 502
Trade Winning: 71.12%
Average Trade: 0.12%
Average Win: 0.94%
Average Loss: -1.90%
Best Trade: 5.48%
Worst Trade: -12.11%
Worst Trade Date: 0 days 02:00:00
Avg Days in Trade: 0 days 01:44:35
---------------------------------
  260 | 02m32s |   -0.39230 | 45.000000 | 65.000000 | 1.000000 | 85.000000 | 120.000000 | 1.000000 | 
---------------------------------

---------------------------------
EMV_RSIStrategy_1_1_120_84_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -5.5047607698
Sortino Ratio: -4.5965203958
Max Drawdown: 0.7288926823
Max Drawdown Duration: 338600
Total Returns: -0.7110830390
Annualized Returns: -1.6735860887
Compound Annual Growth Rate: -0.9461866791
---------------------------------
Trades: 186
Trade Winning: 39.78%
Average Trade: -0.43%
Average Win: 1.77%
Average Loss: -1.89%
Best Trade: 6.91%
Worst Trade: -10.22%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 02:19:21
---------------------------------
  269 | 01m59s |   -0.71108 | 45.000000 | 120.000000 | 1.000000 | 85.000000 | 1.000000 | 84.000000 | 
--------------------------------

---------------------------------
EMV_RSIStrategy_120_52_84_120_55_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.1308127783
Sortino Ratio: 0.0763755708
Max Drawdown: 0.3345262960
Max Drawdown Duration: 168600
Total Returns: -0.0104677143
Annualized Returns: -0.0246365335
Compound Annual Growth Rate: -0.0244622139
---------------------------------
Trades: 6
Trade Winning: 33.33%
Average Trade: 0.49%
Average Win: 11.85%
Average Loss: -5.19%
Best Trade: 12.68%
Worst Trade: -16.13%
Worst Trade Date: 1 days 06:00:00
Avg Days in Trade: 3 days 11:30:00
---------------------------------
  278 | 05m10s |   -0.01047 | 45.000000 | 84.000000 | 52.000000 | 55.000000 | 120.000000 | 120.000000 | 
-----------------------------

---------------------------------
EMV_RSIStrategy_1_1_47_1_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -3.3232305072
Sortino Ratio: -4.1905371640
Max Drawdown: 0.5556268050
Max Drawdown Duration: 349200
Total Returns: -0.5393445647
Annualized Returns: -1.2693869927
Compound Annual Growth Rate: -0.8386640333
---------------------------------
Trades: 477
Trade Winning: 54.93%
Average Trade: 0.05%
Average Win: 1.11%
Average Loss: -1.24%
Best Trade: 7.44%
Worst Trade: -10.22%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 01:03:01
---------------------------------
  287 | 04m38s |   -0.53934 | 45.000000 | 47.000000 | 1.000000 | 85.000000 | 1.000000 | 1.000000 | 
---------------------------------
EMV

...

---------------------------------
EMV_RSIStrategy_55_83_79_77_76_41
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.1193929893
Sortino Ratio: 1.5721119405
Max Drawdown: 0.2638203880
Max Drawdown Duration: 225100
Total Returns: 0.3536562316
Annualized Returns: 0.8323558810
Compound Annual Growth Rate: 1.0394573820
---------------------------------
Trades: 7
Trade Winning: 71.43%
Average Trade: 5.29%
Average Win: 10.38%
Average Loss: -7.44%
Best Trade: 23.55%
Worst Trade: -13.10%
Worst Trade Date: 1 days 08:00:00
Avg Days in Trade: 2 days 18:17:09
---------------------------------
  361 | 00m22s |    0.35366 | 41.000000 | 79.000000 | 83.000000 | 76.000000 | 55.000000 | 77.000000 | 
---------------------------------
EM

---------------------------------
EMV_RSIStrategy_120_26_18_1_55_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.8012489521
Sortino Ratio: -0.8056606223
Max Drawdown: 0.4586161309
Max Drawdown Duration: 293900
Total Returns: -0.2893161537
Annualized Returns: -0.6809267885
Compound Annual Growth Rate: -0.5523798463
---------------------------------
Trades: 491
Trade Winning: 70.26%
Average Trade: 0.15%
Average Win: 0.96%
Average Loss: -1.77%
Best Trade: 7.67%
Worst Trade: -10.61%
Worst Trade Date: 0 days 07:00:00
Avg Days in Trade: 0 days 01:44:36
---------------------------------
  370 | 04m12s |   -0.28932 | 45.000000 | 18.000000 | 26.000000 | 55.000000 | 120.000000 | 1.000000 | 
-------------------------------

---------------------------------
EMV_RSIStrategy_47_43_120_89_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.6820316286
Sortino Ratio: 0.3200856125
Max Drawdown: 0.4996268420
Max Drawdown Duration: 227600
Total Returns: 0.0986356293
Annualized Returns: 0.2321461883
Compound Annual Growth Rate: 0.2478205418
---------------------------------
Trades: 6
Trade Winning: 50.00%
Average Trade: 2.35%
Average Win: 10.41%
Average Loss: -5.71%
Best Trade: 19.97%
Worst Trade: -15.22%
Worst Trade Date: 2 days 02:00:00
Avg Days in Trade: 3 days 17:10:00
---------------------------------
  379 | 04m22s |    0.09864 | 45.000000 | 120.000000 | 43.000000 | 85.000000 | 47.000000 | 89.000000 | 
---------------------------------


---------------------------------
EMV_RSIStrategy_120_1_56_91_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.3517449857
Sortino Ratio: 1.1468377253
Max Drawdown: 0.2649594683
Max Drawdown Duration: 105500
Total Returns: 0.2930748325
Annualized Returns: 0.6897731146
Compound Annual Growth Rate: 0.8311101449
---------------------------------
Trades: 11
Trade Winning: 27.27%
Average Trade: 3.64%
Average Win: 24.61%
Average Loss: -4.22%
Best Trade: 46.30%
Worst Trade: -13.55%
Worst Trade Date: 0 days 15:00:00
Avg Days in Trade: 2 days 06:10:55
---------------------------------
  388 | 04m58s |    0.29307 | 45.000000 | 56.000000 | 1.000000 | 85.000000 | 120.000000 | 91.000000 | 
---------------------------------
E

---------------------------------
EMV_RSIStrategy_91_35_47_72_85_15
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.7419033962
Sortino Ratio: 2.1873146751
Max Drawdown: 0.0530358802
Max Drawdown Duration: 351500
Total Returns: 0.1384130946
Annualized Returns: 0.3257653704
Compound Annual Growth Rate: 0.3567689719
---------------------------------
Trades: 1
Trade Winning: 100.00%
Average Trade: 14.07%
Average Win: 14.07%
Average Loss: nan%
Best Trade: 14.07%
Worst Trade: 14.07%
Worst Trade Date: 5 days 21:00:00
Avg Days in Trade: 5 days 21:00:00
---------------------------------
  397 | 04m50s |    0.13841 | 15.000000 | 47.000000 | 35.000000 | 85.000000 | 91.000000 | 72.000000 | 
---------------------------------
EMV

In [9]:
print(BO.res['max'])

{'max_val': 1.059754296185353, 'max_params': {'window_EMV': 59.0, 'n': 91.0, 'm': 85.0, 'window_RSI': 84.0, 's': 79.0, 'b': 39.0}}


In [10]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

Parameters    Target
328       [1.0, 1.0, 62.0, 65.0, 85.0, 45.0] -0.754406
268      [1.0, 1.0, 120.0, 84.0, 85.0, 45.0] -0.711083
286        [1.0, 1.0, 47.0, 1.0, 85.0, 45.0] -0.539345
395      [1.0, 1.0, 37.0, 120.0, 55.0, 45.0] -0.528045
185    [67.0, 25.0, 119.0, 35.0, 61.0, 44.0] -0.457842
162    [76.0, 102.0, 90.0, 16.0, 64.0, 33.0] -0.440765
364     [87.0, 39.0, 120.0, 1.0, 85.0, 15.0] -0.434585
290      [120.0, 1.0, 75.0, 1.0, 55.0, 15.0] -0.431659
285   [120.0, 120.0, 120.0, 1.0, 85.0, 15.0] -0.429735
320   [120.0, 120.0, 120.0, 1.0, 55.0, 45.0] -0.429735
69     [81.0, 104.0, 75.0, 22.0, 74.0, 43.0] -0.423771
151     [1.0, 56.0, 100.0, 30.0, 55.0, 45.0] -0.418601
306      [1.0, 32.0, 120.0, 1.0, 85.0, 45.0] -0.394370
259      [120.0, 1.0, 65.0, 1.0, 85.0, 45.0] -0.392298
153    [93.0, 56.0, 120.0, 50.0, 85.0, 45.0] -0.388431
107      [1.0, 86.0, 107.0, 1.0, 55.0, 21.0] -0.384166
226    [1.0, 120.0, 120.0, 37.0, 85.0, 45.0] -0.369897
295     [120.0, 1.0, 88.0, 59.0, 55.0, 45.0] -0.366005
35      [14.0, 44.0, 78.0, 18.0, 67.0, 32.0] -0.357359
363       [1.0, 56.0, 32.0, 1.0, 55.0, 15.0] -0.330481
231     [28.0, 25.0, 45.0, 18.0, 65.0, 36.0] -0.325154
193      [1.0, 35.0, 105.0, 1.0, 55.0, 15.0] -0.313748
140     [59.0, 120.0, 72.0, 1.0, 55.0, 15.0] -0.311143
264    [91.0, 120.0, 20.0, 15.0, 55.0, 45.0] -0.298659
0       [38.0, 24.0, 56.0, 48.0, 62.0, 40.0] -0.290861
381     [51.0, 82.0, 39.0, 27.0, 55.0, 45.0] -0.290585
369     [120.0, 26.0, 18.0, 1.0, 55.0, 45.0] -0.289316
211   [120.0, 120.0, 51.0, 65.0, 79.0, 45.0] -0.279035
190    [42.0, 14.0, 101.0, 50.0, 59.0, 44.0] -0.278516
317      [62.0, 59.0, 77.0, 1.0, 85.0, 15.0] -0.272330
..                                       ...       ...
265   [44.0, 35.0, 120.0, 101.0, 55.0, 45.0]  0.302624
147      [44.0, 69.0, 27.0, 1.0, 55.0, 45.0]  0.304706
11      [77.0, 33.0, 14.0, 84.0, 60.0, 41.0]  0.307464
96       [9.0, 54.0, 87.0, 13.0, 61.0, 31.0]  0.320254
350   [50.0, 25.0, 120.0, 120.0, 85.0, 45.0]  0.333577
236     [53.0, 48.0, 55.0, 11.0, 82.0, 35.0]  0.343259
382     [73.0, 87.0, 65.0, 92.0, 76.0, 43.0]  0.345551
243     [54.0, 82.0, 80.0, 75.0, 80.0, 39.0]  0.348088
109    [24.0, 102.0, 47.0, 62.0, 60.0, 31.0]  0.350977
360     [55.0, 83.0, 79.0, 77.0, 76.0, 41.0]  0.353656
131     [99.0, 1.0, 36.0, 120.0, 55.0, 45.0]  0.362702
371      [44.0, 1.0, 120.0, 1.0, 85.0, 15.0]  0.365991
133     [80.0, 57.0, 73.0, 56.0, 80.0, 32.0]  0.398327
189   [45.0, 96.0, 120.0, 104.0, 85.0, 45.0]  0.417681
119     [33.0, 63.0, 18.0, 13.0, 80.0, 42.0]  0.419529
45     [61.0, 83.0, 117.0, 71.0, 78.0, 43.0]  0.431675
168     [26.0, 73.0, 47.0, 33.0, 61.0, 44.0]  0.438412
16    [51.0, 114.0, 78.0, 104.0, 79.0, 43.0]  0.468560
274     [48.0, 97.0, 88.0, 73.0, 80.0, 42.0]  0.508973
297     [60.0, 91.0, 83.0, 79.0, 79.0, 42.0]  0.527245
235     [65.0, 96.0, 77.0, 79.0, 78.0, 41.0]  0.537540
241    [57.0, 79.0, 117.0, 69.0, 73.0, 41.0]  0.538949
191    [49.0, 30.0, 102.0, 81.0, 67.0, 39.0]  0.540559
245     [18.0, 1.0, 120.0, 99.0, 55.0, 45.0]  0.570948
60     [58.0, 61.0, 49.0, 113.0, 75.0, 42.0]  0.583734
230     [57.0, 89.0, 87.0, 88.0, 84.0, 43.0]  0.688239
302     [54.0, 85.0, 85.0, 85.0, 77.0, 37.0]  0.745347
112  [55.0, 120.0, 120.0, 120.0, 85.0, 45.0]  0.792312
195  [52.0, 118.0, 110.0, 115.0, 81.0, 44.0]  0.861594
216     [59.0, 91.0, 85.0, 84.0, 79.0, 39.0]  1.059754

[400 rows x 2 columns]

## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)